# Import main library and run test experiment

Test that the library is correctly installed in the current virtual environment by doing a global import. Then, test the basic functionality by executing a test run with dummy data.

In order to install the library locally, navigate to its root folder and run the following command:

    pip install -e .
    
For running the simulation, we can either load a configuration file in the config folder, or write the configuration manually and adding it to the simulation handler

## Method 1: Using a configuration file

In [1]:
import spikingFT.startup
import matplotlib.pyplot as plt
import numpy as np

In [2]:
conf_file = "../config/test_radix4.json"
sim_handler = spikingFT.startup.startup(conf_file)

INFO: Running spiking-FT:
- Configuration file: ../config/test_radix4.json
- FT mode: fft
- Framework: radix4loihi
- Test performance: False
- Nº Samples: 64
INFO: Loading data
INFO: Data loaded:
- Source: BBM
- Nº frames: 1
- Nº chirps: 1
- Nº samples: 64
INFO: Encoding data to spikes
INFO: Initializing SNN simulation
DEBUG: Creating Compartments ...
DEBUG: Creating CompartmentPrototypes of Layer 0 ...
DEBUG: Creating CompartmentPrototypes of Layer 1 ...
DEBUG: Creating CompartmentPrototypes of Layer 2 ...
DEBUG: Done.
DEBUG: Creating ConnectionPrototype ...
DEBUG: Creating auxillary neurons ...
DEBUG: Clock neuron connected to layer 0.
DEBUG: Reset neuron connected to layer 0.
DEBUG: Clock neuron connected to layer 1.
DEBUG: Reset neuron connected to layer 1.
DEBUG: Clock neuron connected to layer 2.
DEBUG: Reset neuron connected to layer 2.
DEBUG: Done.
DEBUG: Creating Probes ...
DEBUG: Creating Probes of Layer 0 ...
DEBUG: Creating Probes of Layer 1 ...
DEBUG: Creating Probes of La

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 10.212.98.110:40293
INFO:DRV:      Host server up..............Done 0.22s
INFO:DRV:      Encoding axons/synapses.....Done 3.04s
INFO:DRV:      Compiling Embedded snips....Done 0.84s
INFO:DRV:      Compiling MPDS Registers....Done 1.10ms
INFO:HST:  Args chip=0 cpu=0 /homes/nreeb/loihi/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1632236606.0862906/launcher_chip0_lmt0.bin --chips=3 --epoch=0 --remote-relay=1 
INFO:HST:  Args chip=1 cpu=0 /homes/nreeb/loihi/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1632236606.0862906/launcher_chip1_lmt0.bin --chips=3 --epoch=0 --remote-relay=1 
INFO:HST:  Args chip=2 cpu=0 /homes/nreeb/loihi/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1632236606.0862906/launcher_chip2_lmt0.bin --chips=3 --epoch=0 --remote-relay=1 
INFO:HST:  Args chip=3 cpu=0 /homes/nreeb/loihi/lib/python3.8/site-packages/nxsdk/driver/../bin/arm/idle_chip.bi

DEBUG: Done.
DEBUG: Running simulation ... 


INFO:DRV:      Transferring probes.........Done 0.01s
INFO:DRV:      Configuring registers.......Done 0.57s
INFO:DRV:      Transferring spikes.........Done 0.01s
INFO:HST:  chip=3 cpu=0 halted, status=0x0
INFO:HST:  srun: interrupt (one more within 1 sec to abort)
INFO:HST:  srun: step:1246529.0 task 0: running
INFO:DRV:      Executing...................Done 291.72s
INFO:DRV:  Executor: 40 timesteps..........Done 292.33s


KeyboardInterrupt: 

In [ ]:
chirp_data = sim.encoded_data[0,:]
fft = np.abs(np.fft.fft(chirp_data))
fig = plt.plot(fft[1:int(fft.size/2)])

## Method 2: Setting the configuration manually

In [1]:
import spikingFT.sim_handler
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib

In [2]:
path = "data/BBM"

config = {}
config["snn_config"] = {
    "sim_time": 64,
    "mode": "fft",
    "framework": "radix4loihi",
    "current_decay": 0,
    "measure_performance": False
}
config["data"] = {
    "samples_per_chirp": 64,
    "chirps_per_frame": 1,
    "antennas": 1,
    "nframes": 1
}

### Instantiate the simulation handler and run the simulation with the given configuration

In [3]:
datapath = pathlib.Path('').resolve().parent.joinpath(path)
sim_handler = spikingFT.sim_handler.SimHandler(datapath, config)
sim_handler.run()

AttributeError: 'NoneType' object has no attribute 'run'

### Collect the output data from the SNN and plot simulation output

The output is stored in the voltage probes and in the spiking probes, which are hanging from the simulator handler.
If the configuration of the network was good, it should be possible to observe the following patterns in the voltage plot:

* A clear change from the charging stage to the spiking stage at t=sim_time
* A majority of neurons ending the charging stage around 0V and spiking at around sim_time*1.5
* The offset neuron typically has a very high value and spikes very soon. This value is filtered out in the final result plot

In [ ]:
config = sim_handler.config
sim_time = config["snn_config"]["sim_time"]
nsamples = config["data"]["samples_per_chirp"]

plt.figure(2, figsize=(15,15))
ax1 = plt.subplot(2, 1, 1)

v_plots_re = sim_handler.snn.l1_real_probes_V[0].plot()
v_plots_im = sim_handler.snn.l1_imag_probes_V[0].plot()
plt.title('Voltage over time')

cr = []
ci = []
for i in range(nsamples):
    cr.append(v_plots_re[i].get_color())
    ci.append(v_plots_im[i].get_color())

ax2 = plt.subplot(2, 1, 2)
sr = sim_handler.snn.l1_real_probes_S[0].plot(colors=cr)
si = sim_handler.snn.l1_imag_probes_S[0].plot(colors=ci)

plt.title('Spikes over time')

### Plot S-FT final result

Combine the spiking times of the neurons representing the real and imaginary terms of the FT, so the modulus can be plotted.
Most of the values should be laying around zero. If that is not the case, the time offset removal is faulty. Check in the previous step that neurons representing values around zero spike at around sim_time*1.5

In [ ]:
sr = []
si = []

for i in range(1, nsamples):
    spike_data = np.array(sim_handler.snn.l1_real_probes_S[0][i].data)
    spike_t = np.where(spike_data==1)
    single_spike_t = spike_t[0][0] if spike_t[0].size != 0 else sim_time*1.5
    sr.append(single_spike_t-sim_time*1.5)
    
for i in range(1, nsamples):
    spike_data = np.array(sim_handler.snn.l1_imag_probes_S[0][i].data)
    spike_t = np.where(spike_data==1)
    single_spike_t = spike_t[0][0] if spike_t[0].size != 0 else sim_time*1.5
    si.append(single_spike_t-sim_time*1.5)
    
plt.figure()
plt.title('SDFT Result')
sdft = np.sqrt(np.array(sr)**2+np.array(si)**2)
result = np.split(sdft ,[int(nsamples/2),int(nsamples/2)])

plt.plot(result[0])

### Show the ideal output of the DFT (computed in numpy)

In [ ]:
chirp_data = sim_handler.encoded_data[0, :]
fft = np.abs(np.fft.fft(chirp_data))
plt.plot(fft[1:int(fft.size/2)])
